# Missouri prison records

This notebook has code to download, unzip and parse a fixed-width file of [inmates held by the Missouri Department of Corrections](https://doc.mo.gov/Sunshine_Law/) and load it into a pandas dataframe. The department also releases a handful of XML files with column specs, field descriptions and lookups for facility locations and county abbreviations.

No analysis, yet -- just loading it up.

### Import libraries

In [1]:
import xml.etree.ElementTree as ET
from zipfile import ZipFile
from collections import OrderedDict

import requests
from slugify import slugify
import pandas as pd
import numpy as np

### The download function

In [2]:
def download_files():
    files = ['https://doc.mo.gov/xml/sunshine_layout.xml',
             'https://doc.mo.gov/xml/sunshine_desc.xml',
             'https://doc.mo.gov/xml/sunshine_locations.xml',
             'https://doc.mo.gov/xml/sunshine_counties.xml',
             'https://doc.mo.gov/Sunshine_Law/fak930.zip']
    
    for f in files:
        r = requests.get(f)
        r.raise_for_status()

        filename = f.split('/')[-1]

        with open(filename, 'wb') as f:
            for block in r.iter_content(1024):
                f.write(block)
    
    return [x.split('/')[-1] for x in files]

In [ ]:
'''
to download the files and unzip the main data file,
un-comment the lines in this cell and then run it
'''

# download_files()
# with ZipFile('fak930.zip', 'r') as z:
#    z.extractall('.')

### Function to parse the county abbreviation file

This function parses the county XML file and returns a dictionary.

In [4]:
def county_dict(infile='sunshine_counties.xml'):
    tree = ET.parse(infile)
    locations = tree.getroot()
    
    return {x.find('acronym').text: x.find('fullname').text
            for x in locations}

### Function to parse the file layout XML

This function parses the XML with the file layout. It returns an OrderedDict where each key is a slugified column name and each value is a `(start_position, end_position)` tuple -- the format we'll need when we pass the values to the `colspecs` argument of pandas' [`read_fwf()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_fwf.html) method.

Here's an example of an item in the OrderedDict that gets returned: `('doc-id', (0, 8))`

In [5]:
def layout_dict(infile='sunshine_layout.xml'):

    tree = ET.parse(infile)
    locations = tree.getroot()

    counter = 0
    d = OrderedDict()
    for l in locations:
        desc = slugify(l.find('desc').text, to_lower=True)
        start = counter
        data_len = int(l.find('length').text.split(',')[0].strip())
        end = start + data_len
        d[desc] = (start, end)
        counter += data_len
    
    return d

### Function to parse the locations XML

This function parses the locations XML file and returns a dictionary.

In [6]:
def location_dict(infile='sunshine_locations.xml'):
    tree = ET.parse(infile)
    locations = tree.getroot()

    return {x.find('acronym').text: x.find('name').text
            for x in locations}

### Get the column names and colspecs for our data frame

First, we parse the file layout. The data file doesn't have headers, so we'll extract that list from the dictionary keys. The colspecs, in turn, are the values.

In [7]:
file_layout = layout_dict()
columns = list(file_layout.keys())
colspecs = list(file_layout.values())

### Function to parse dates

The dates in the data are formatted `YYYYMMDD`. Or they're nonsense dates (99999999), or null, or 0. So we'll try to parse the date in that format; if not, return `NaN`.

In [8]:
def dateparse(x):
    try:
        return pd.datetime.strptime(x, '%Y%m%d')
    except (TypeError, ValueError):
        return np.NaN

### Read the data into a data frame

... using the `read_fwf()` method. Apart from the file name, we're handing this method six additional arguments:
- `header=None`: The data file doesn't have a header row
- `names=columns`: The column names
- `colspecs=colspecs`: The list of tuples with start/end positions for each break
- `dtypes={'doc-id': str}`: Interpret the first column (a number with leading zeroes) as a string
- `parse_dates=[...]`: The columns to parse as dates
- `date_parser=dateparse`: Use the `dateparser` function, defined above, to handle dates

In [9]:
df = pd.read_fwf('SunshineLawFak930',
                 header=None,
                 names=columns,
                 colspecs=colspecs,
                 dtypes={'doc-id': str},
                 parse_dates=['birth-date', 'sentence-i-date', 'sentence-maximum-release-date',
                              'sentence-i-minimum-release-date', 'sentence-f-probation-date'],
                 date_parser=dateparse)

In [15]:
df.tail()

,doc-id,offender-last-name,offender-first-name,offender-middle-name,generation-suffix,interface-race-description,interface-sex-description,birth-date,offender-assigned-place,doc-loc-func-flag,...,sentence-maximum-release-date,sentence-i-minimum-release-date,sentence-i-length-years,sentence-i-length-months,sentence-i-length-days,sentence-f-probation-date,probation-type,probation-term-yrs,probation-term-months,probation-term-days
1075064,1327750,WALKER,REBECCA,NaN,NaN,Black,Female,1986-08-29,23,F,...,2023-04-24 00:00:00,NaN,4,0,0,2018-04-25,SES,5,0,0
1075065,1327753,OTT,MARTY,B,NaN,White,Male,1972-02-12,44,F,...,2023-04-23 00:00:00,NaN,4,0,0,2018-04-24,SES,5,0,0
1075066,1327757,BARTLETT,TRAVIS,A,NaN,White,Male,1987-09-11,26,F,...,2020-04-18 00:00:00,NaN,0,6,0,2018-04-19,SES,2,0,0
1075067,1327757,BARTLETT,TRAVIS,A,NaN,White,Male,1987-09-11,26,F,...,2020-04-18 00:00:00,NaN,0,6,0,2018-04-19,SES,2,0,0
1075068,1327785,MCKOY,COURTNEY,O,NaN,Black,Male,1980-11-28,35,F,...,2023-04-22 00:00:00,NaN,4,0,0,2018-04-23,SES,5,0,0
